#### Loading packages

In [1]:
import torch
from astropy.io import fits
import glob
import numpy as np
import pandas as pd
import time

In [2]:
if (torch.cuda.is_available()):
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

# intel HD does not support CUDA

cpu


In [3]:
# classical approach to compare with ANN
def rolo_b_me(alpha, emission, incidence):
    '''
    Here we obtain the reflectance for any photometric angle (using degrees as input)  
    according to our classic photometric modeling. This model consists of the Sommel-Seeliger 
    disk function and ROLO as a phase function.

    Use example: for phase=30º, emission=0º and incidence=30º - ipwg(30, 0, 30)
    
    Inputs: phase, emission and incidence angles
    Output: reflectance
    '''
    # parameters
    C_0, C_1, A_0, A_1, A_2, A_3, A_4=np.loadtxt('parameters.txt')  
    print(len(alpha))
    # converting degrees to radians:
    emission=np.deg2rad(emission)    
    incidence=np.deg2rad(incidence)
    
    # computing reflectance according to this photometric model:
    disk = (np.cos(incidence)/(np.cos(incidence)+np.cos(emission)))
    phase = C_0*np.exp(-C_1*alpha) + A_0 + A_1*alpha + A_2*(alpha)**2 + A_3*(alpha)**3  + A_4*(alpha)**4 
    
    reflectance = disk * phase
    
    return reflectance

In [4]:
def data_preparation(ID):
    '''
     Here we extract features and labels using the ID of each image.
    Later we convert these data in the apropiate format for training or validation.
    It calls to data_normalization() function.
    '''
    
    iof_n, phase_n, emission_n, incidence_n = data_normalization(ID)
    
    
    # packing data
    features = np.zeros((len(phase_n),3)) 
    features[:,0] = phase_n
    features[:,1] = emission_n
    features[:,2] = incidence_n
    
    features = torch.FloatTensor(features)
    features = features.to(device)
    
    labels = np.zeros((len(iof_n),1))
    labels[:,0] = iof_n
    
    labels = torch.FloatTensor(labels)
    labels = labels.to(device)
    
    return features, labels

In [5]:
def data_normalization(ID):
    '''
   In this function we rescale data between 0 and 1, a very important step before trainning a neural network.
    It calls to loading_and_cleaning_data() function.
    '''
    iof_, phase_, emission_, incidence_ = loading_and_cleaning_data(ID)
       
    #normalizing data
    iof_n=iof_/0.06; phase_n=phase_/90; emission_n=emission_/82; incidence_n=incidence_/82

    return iof_n, phase_n, emission_n, incidence_n

In [6]:
def loading_and_cleaning_data(ID):
    '''
    This function is for loading iof, phase, emission, incidence values for each image. 
    '''
    #criterion for removing data:
    em, eM = (0, 82)        # emission limits  
    im, iM = (0, 82)          # incidence limits
    pm, pM = (0, 90)       # phase limits
    rm, rM =  (0.001,1)  # reflectance higher limits
    
    fits_file = fits.getdata(ID, ignore_missing_end=True)
    
    iof = fits_file[0]
    phase = fits_file[1]
    emission = fits_file[2]
    incidence = fits_file[3]
    
    idxsort = (emission >= em) & (emission <= eM) & \
              (incidence >= im) & (incidence <= iM) & \
              (phase >= pm) & (phase <= pM) & \
              (iof >= rm) & (iof <= rM) 
    
    return iof[idxsort], phase[idxsort], emission[idxsort], incidence[idxsort] 

In [7]:
# loading ID list of files:
files_ID=[]
for i in sorted(glob.glob('reduced_phocubes/*reduce.fits')):
    files_ID.append(i)

In [8]:
len(files_ID)

951

#### Preparing training and validation data

In [9]:
import random
# defining global tensors 

final_training_features = torch.zeros(0)
final_training_labels = torch.zeros(0)
final_validation_features = torch.zeros(0)
final_validation_labels = torch.zeros(0)

final_training_features = final_training_features.to(device)
final_training_labels = final_training_labels.to(device)
final_validation_features = final_validation_features.to(device)
final_validation_labels = final_validation_labels.to(device)

for file in files_ID:
    
    # loading training data from files_ID list
    _features, _labels = data_preparation(file)
    if len(_features) != 0:
        np.random.seed(10)
        # Selecting 1000 random values of this image
        try:
            mask_temp = np.random.choice(len(_features),1000,replace=False)
        
        except:
            mask_temp = np.random.choice(len(_features),len(_features),replace=False)
        
        split = int(len(mask_temp)*(9/10))
        mask_train = mask_temp[0:split]
        mask_val = mask_temp[split:]
        
        training_features =  _features[mask_train]
        final_training_features = torch.cat([final_training_features,training_features])
        
        training_labels = _labels[mask_train]
        final_training_labels = torch.cat([final_training_labels,training_labels])
        
        validation_features =  _features[mask_val]
        final_validation_features=torch.cat([final_validation_features,validation_features])

        validation_labels = _labels[mask_val]
        final_validation_labels=torch.cat([final_validation_labels,validation_labels])

In [10]:
#ROLO eficciency using MSE:
start_time = time.time()
phase=final_validation_features.cpu().numpy()[:,0]*90
emission=final_validation_features.cpu().numpy()[:,1]*82
incidence=final_validation_features.cpu().numpy()[:,2]*82
prediction_ROLO=torch.tensor(rolo_b_me(phase, emission, incidence)/0.06).to(device)
eff_ROLO=torch.mean((prediction_ROLO-final_validation_labels.t())**2)
print(eff_ROLO)

46480
tensor(0.0044)


In [11]:
print(len(final_training_features), len(final_validation_features))
final_training_features = final_training_features.numpy()
final_validation_features = final_validation_features.numpy()
final_training_labels = final_training_labels.numpy()
final_validation_labels = final_validation_labels.numpy()

417025 46480


In [12]:
import tensorflow as tf
input_size = 3
final_training_labels = tf.convert_to_tensor(final_training_labels)
final_validation_lables =  tf.convert_to_tensor(final_validation_labels)

#### Setting up and defining a neural network

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy

numpy.random.seed(7)

model = Sequential()
model.add(Dense(15, input_shape=(input_size,), activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam')

modelfit = model.fit(final_training_features, final_training_labels, 
                     validation_data=(final_validation_features, final_validation_labels), epochs=20, batch_size=200)

best_loss = modelfit.history['val_loss'][-1]


Epoch 1/20
2086/2086 [==============================] - 5s 2ms/step - loss: 0.0056 - val_loss: 0.0038
Epoch 2/20
2086/2086 [==============================] - 4s 2ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 3/20
2086/2086 [==============================] - 4s 2ms/step - loss: 0.0037 - val_loss: 0.0038
Epoch 4/20
2086/2086 [==============================] - 5s 2ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 5/20
2086/2086 [==============================] - 17s 8ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 6/20
2086/2086 [==============================] - 8s 4ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 7/20
2086/2086 [==============================] - 5s 3ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 8/20
2086/2086 [==============================] - 3s 2ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 9/20
2086/2086 [==============================] - 4s 2ms/step - loss: 0.0036 - val_loss: 0.0038
Epoch 10/20
2086/2086 [==============================] - 3s 2ms/step - loss: 0.00

In [14]:
print(eff_ROLO)
print(best_loss)

tensor(0.0044)
0.0035880310460925102


#### Improvement percentage

In [15]:
print((eff_ROLO - best_loss)*100/eff_ROLO)

tensor(18.4758)
